In [1]:
import sys
sys.path.insert(1, '/home/georgeos/Documents/GitHub/SyMBac/')

from SyMBac.cell_simulation import Simulation
from SyMBac.PSF import PSF_generator
from SyMBac.drawing import Renderer
from SyMBac.PSF import Camera
from SyMBac.misc import get_sample_images
real_image = get_sample_images()["E. coli 100x"]

/home/georgeos/Documents/GitHub/SyMBac/SyMBac/cell_simulation.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [2]:
my_simulation = Simulation(
    trench_length=15,
    trench_width=1.3,
    cell_max_length=6.65, #6, long cells # 1.65 short cells
    cell_width= 1, #1 long cells # 0.95 short cells
    sim_length = 100,
    pix_mic_conv = 0.065,
    gravity=0,
    phys_iters=15,
    max_length_var = 0.,
    width_var = 0.,
    lysis_p = 0.,
    save_dir="/tmp/",
    resize_amount = 3
)

In [3]:
my_simulation.run_simulation(show_window=False)
my_simulation.draw_simulation_OPL(do_transformation=True, label_masks=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Timeseries Properties:   0%|          | 0/98 [00:00<?, ?it/s]

Scene Draw::   0%|          | 0/98 [00:00<?, ?it/s]

In [4]:
my_kernel = PSF_generator(
    radius = 50, 
    wavelength = 0.75, 
    NA = 1.2, 
    n = 1.3, 
    resize_amount = 3, 
    pix_mic_conv = 0.065, 
    apo_sigma = 10, 
    mode="phase contrast", 
    condenser = "Ph3")
my_kernel.calculate_PSF()

In [5]:
my_camera = Camera(baseline=100, sensitivity=2.9, dark_noise=8)

In [6]:
my_renderer = Renderer(my_simulation, my_kernel, real_image, my_camera)

In [7]:
my_renderer.select_intensity_napari()

In [8]:
my_renderer.optimise_synth_image(manual_update=False)

/home/georgeos/Documents/GitHub/SyMBac/SyMBac/drawing.py:942: RuntimeWarning: Mean of empty slice.
  self.real_media_mean = self.real_resize[np.where(self.media_label.data)].mean()
/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/georgeos/Documents/GitHub/SyMBac/SyMBac/drawing.py:943: RuntimeWarning: Mean of empty slice.
  self.real_cell_mean = self.real_resize[np.where(self.cell_label.data)].mean()
/home/georgeos/Documents/GitHub/SyMBac/SyMBac/drawing.py:944: RuntimeWarning: Mean of empty slice.
  self.real_device_mean = self.real_resize[np.where(self.device_label.data)].mean()
/home/georgeos/Documents/GitHub/SyMBac/SyMBac/drawing.py:947: RuntimeWarning: Degrees of freedom <= 0 for slice
  self.real_media_var = self.real_resize[np.where(self.media_label.data)].var()
/home/georgeos/miniconda3/envs/symbac/lib/python3.9/site-packages/numpy/co

interactive(children=(IntSlider(value=75, description='media_multiplier', max=300, min=-300), FloatSlider(valu…

In [9]:
my_renderer.generate_training_data(sample_amount=0.2, randomise_hist_match=True, randomise_noise_match=True, burn_in=40, n_samples = 500, save_dir="/tmp/test/", in_series=False)

Sample generation: 100%|██████████████████████| 500/500 [00:44<00:00, 11.25it/s]
